In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold
from sklearn.metrics import roc_curve

In [2]:
# !pip install numpy==1.16.5  # 1.19.5
# !pip install Cython==0.27.3

In [3]:
from autosklearn.classification import AutoSklearnClassifier

/Users/ellenhoeven/anaconda3/envs/wids2021/lib/python3.8/site-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


# Import data and very quick preprocessing
- Create target and features
- Drop columns that are not needed
- Encode categorical columns

In [4]:
data = pd.read_csv("TrainingWiDS2021.csv")

In [5]:
data.head()

,Unnamed: 0,encounter_id,hospital_id,age,bmi,elective_surgery,ethnicity,gender,height,hospital_admit_source,...,h1_pao2fio2ratio_max,h1_pao2fio2ratio_min,aids,cirrhosis,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,diabetes_mellitus
0,1,214826,118,68.0,22.732803,0,Caucasian,M,180.3,Floor,...,NaN,NaN,0,0,0,0,0,0,0,1
1,2,246060,81,77.0,27.421875,0,Caucasian,F,160.0,Floor,...,51.0,51.0,0,0,0,0,0,0,0,1
2,3,276985,118,25.0,31.952749,0,Caucasian,F,172.7,Emergency Department,...,NaN,NaN,0,0,0,0,0,0,0,0
3,4,262220,118,81.0,22.635548,1,Caucasian,F,165.1,Operating Room,...,337.0,337.0,0,0,0,0,0,0,0,0
4,5,201746,33,19.0,NaN,0,Caucasian,M,188.0,NaN,...,NaN,NaN,0,0,0,0,0,0,0,0


In [6]:
y = data['diabetes_mellitus']

In [7]:
X = data.drop(['diabetes_mellitus'], axis=1)

In [8]:
X.drop("Unnamed: 0", axis = 1, inplace = True)
X.drop("encounter_id", axis = 1, inplace = True)
X.drop("hospital_id", axis = 1, inplace = True)
X.drop("icu_id", axis = 1, inplace = True)
X.drop("icu_stay_type", axis = 1, inplace = True)

In [9]:
categorical_cols = ['elective_surgery',
                   'ethnicity',
                   'gender',
                   'hospital_admit_source',
                   'icu_admit_source',
                   'icu_type',
                   'readmission_status']

In [10]:
X = pd.get_dummies(X, columns=categorical_cols)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.75, test_size=0.25, 
                                                    random_state=42)

# Start auto-sklearn

https://machinelearningmastery.com/auto-sklearn-for-automated-machine-learning-in-python/

https://automl.github.io/auto-sklearn/master/examples/20_basic/example_classification.html#sphx-glr-examples-20-basic-example-classification-py


In [12]:
# define search
model = AutoSklearnClassifier(time_left_for_this_task=5*60, per_run_time_limit=30, n_jobs=8)
# perform the search
model.fit(X_train, y_train)
# summarize
print(model.sprint_statistics())

auto-sklearn results:
  Dataset name: bb577bd6e3d6596077995f0c46ef35c3
  Metric: accuracy
  Best validation score: 0.814770
  Number of target algorithm runs: 54
  Number of successful target algorithm runs: 27
  Number of crashed target algorithm runs: 2
  Number of target algorithms that exceeded the time limit: 25
  Number of target algorithms that exceeded the memory limit: 0



In [13]:
# evaluate best model
y_hat = model.predict(X_test)
roc = roc_curve(y_test, y_hat)

In [14]:
dir(model)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_n_features',
 '_get_automl_class',
 '_get_param_names',
 '_get_tags',
 '_load_models',
 '_metric',
 '_more_tags',
 '_n_jobs',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_scoring_functions',
 '_validate_data',
 'automl_',
 'build_automl',
 'cv_results_',
 'dask_client',
 'delete_output_folder_after_terminate',
 'delete_tmp_folder_after_terminate',
 'disable_evaluator_output',
 'ensemble_nbest',
 'ensemble_size',
 'exclude_estimators',
 'exclude_preprocessors',
 'fANOVA_input_',
 'fit',
 'fit_ensemble',
 'get_configuration_space',
 'get_models_with_weights',
 'get_

In [15]:
model.show_models()

"[(0.400000, SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'lda', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'quantile_transformer', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:lda:shrinkage': 'manual', 'classifier:lda:tol': 0.08263223933160525, 'data_preprocessing:categorical_transformer:category_coalescence:minority_coalescer:minimum_fraction': 0.053754924476457716, 'data_preprocessing:numerical_transformer:rescaling:quantile_transformer:n_quantiles': 1778, 'data_preprocessing:numerical_transformer:rescaling:quantile_transformer:output_distribution': 'uniform', 'classifier:lda:shrinkage_factor': 0.5576239002938481},\ndataset_properties=

In [16]:
model.cv_results_

{'mean_test_score': array([0.        , 0.78571429, 0.        , 0.        , 0.        ,
        0.71841435, 0.        , 0.        , 0.78105793, 0.        ,
        0.        , 0.76680946, 0.80374992, 0.        , 0.        ,
        0.        , 0.75423729, 0.79701372, 0.81374558, 0.        ,
        0.77053455, 0.        , 0.78323089, 0.80278761, 0.80744397,
        0.78375861, 0.        , 0.80672999, 0.78375861, 0.        ,
        0.80781648, 0.        , 0.78375861, 0.        , 0.        ,
        0.        , 0.79561681, 0.        , 0.81476998, 0.        ,
        0.        , 0.        , 0.        , 0.80086298, 0.        ,
        0.80555038, 0.78375861, 0.80977215, 0.78844602, 0.        ,
        0.79757248, 0.80362575, 0.80806482, 0.        ]),
 'mean_fit_time': array([30.1250298 , 25.11856103, 30.1183238 , 30.11058259, 30.10567379,
        27.03911281, 27.93290401, 30.09300709, 17.67097783, 30.09580016,
        30.08211875, 28.14452767, 26.11144209, 30.17468286, 30.14534307,
       